In [1]:
def correlation_2D(step):
    abs_n = "FFT_" + "N_" + str(step) + ".txt"
    abs_p = "FFT_" + "P_" + str(step) + ".txt"
    mod_n = np.loadtxt(abs_n, float)
    mod_p = np.loadtxt(abs_p, float)

    n2_inv = np.fft.ifftn(mod_n**2)
    max_fft_n2_inv = np.where(n2_inv == n2_inv.max())
    n2_inv[max_fft_n2_inv[0][0],max_fft_n2_inv[1][0]] = 0   
    
    p2_inv = np.fft.ifftn(mod_p**2)
    max_fft_p2_inv = np.where(p2_inv == p2_inv.max())
    p2_inv[max_fft_p2_inv[0][0],max_fft_p2_inv[1][0]] = 0
    
    np_inv = np.fft.ifftn(mod_n*mod_p)
    max_fft_np_inv = np.where(np_inv == np_inv.max())
    np_inv[max_fft_np_inv[0][0],max_fft_np_inv[1][0]] = 0
 
    n2_inv = np.fft.fftshift(n2_inv)
    p2_inv = np.fft.fftshift(p2_inv)
    np_inv = np.fft.fftshift(np_inv)

    file_abs_n  = np.abs(n2_inv)
    file_abs_p  = np.abs(p2_inv)
    file_abs_np = np.abs(np_inv)

    file_n_real = "matrix_" + "N" + "_" + str(step) + "_nb_1.txt"
    file_p_real = "matrix_" + "P" + "_" + str(step) + "_nb_1.txt"

    file_load_n_real = np.loadtxt(file_n_real, float)
    file_load_p_real = np.loadtxt(file_p_real, float)

    mean_n = np.mean(file_load_n_real)
    mean_p = np.mean(file_load_p_real)
    
    std_n = np.std(n2_inv)
    std_p = np.std(p2_inv)

    corr_nn = file_abs_n - mean_n**2
    test_nn = np.corrcoef(mod_n)

    corr_pp = file_abs_p - mean_p**2
    corr_np = file_abs_np*file_abs_np - mean_n*mean_p
    
    return corr_nn, corr_pp, corr_np, test_nn

In [ ]:
def plot_correlation_matrix(matrix_1, matrix_2, matrix_3, t): 
    plt.figure(figsize=(20, 3))
    
    plt.subplot(141)
    plt.imshow(matrix_1, cmap = 'seismic' , interpolation = 'bilinear')
    plt.title("$corr\_nn(t={})$".format(t))
    plt.colorbar()
    
    plt.subplot(142)
    plt.imshow(matrix_2, cmap = 'seismic' , interpolation = 'bilinear')
    plt.title("$corr\_pp(t={})$".format(t))
    plt.colorbar()

    plt.subplot(143)
    plt.imshow(matrix_2, cmap = 'seismic' , interpolation = 'bilinear')
    plt.title("$corr\_np(t={})$".format(t))
    plt.colorbar()

    plt.tight_layout(pad=-3.0)
    plt.show()


In [ ]:
def polar_mean(step, corr_type):
    matrix_file = "matrix_corr_" + corr_type + "_" + str(step) + ".txt"
    matrix = np.loadtxt(matrix_file, float)
    n = len(matrix)  
    dk = 1.0
    #dk = 1.0*len(matrix)
    #dk = 2.0*np.pi/(len(matrix)*1)
    p = np.zeros(int(n/2))
    for i in range(1,int(n/2)): 
        k = dk*i
        suma = 0.
        dtheta = np.pi/90. # two degrees
        
        for j in range (1,180):
            theta = j * dtheta    
            nx = int(k*np.cos(theta)/dk - n/2)
            ny = int(k*np.sin(theta)/dk - n/2)
            suma += matrix[nx,ny]        
        p[i] = suma/180.
        
    filename = "correlation_" + corr_type + "_" + str(step) + ".txt"         
    np.savetxt(filename, p, delimiter="\n")

In [ ]:
def graph_correlation(step, corr_type):
    file = "correlation_"+corr_type+"_"+str(step)+".txt"
    corr = np.loadtxt(file, float)
    
    plt.plot(corr, "-o")
    plt.title("C_{}(t={})".format(corr_type,step))
    plt.xlabel("r")
    plt.ylabel("C_{}(r)".format(corr_type))

In [ ]:
def plot_correlations_3d(corr_type, time_steps):
    fig = plt.figure(figsize=(15,15))
    ax = plt.axes(projection='3d')

    # plot a 3D surface like in the example mplot3d/surface3d_demo
    X = np.arange(0, 35, 1)
    Y = np.arange(30, time_steps, 5)
    X, Y = np.meshgrid(X, Y)
    Z = np.array([get_correlation(i, corr_type) for i in range(30, time_steps, 5)])

    surf = ax.plot_surface(X, Y, Z, rstride=1, cstride=1, alpha=1, cmap='Reds', linewidth=0.5, edgecolors='black')
    fig.colorbar(surf, shrink=0.3, aspect=8, location='left', ax=ax)

    ax.view_init(15, -20)
    
    ax.set_zlim(bottom=0)
    ax.set_xlim(0,35)
    ax.set_ylim(30,time_steps)

    ax.set_xlabel('Number wave "k"', fontsize=15)
    ax.set_ylabel('Time step', fontsize=15)
    ax.set_zlabel('P(k,t)', fontsize=15)

    return plt.show()

In [ ]:
def get_correlation(step, corr_type):
    file = "correlation_"+corr_type+"_"+str(step)+".txt"
    corr = np.loadtxt(file, float)
    return corr